In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
from random import uniform
import speech_recognition as sr
import ffmpy
import requests
import urllib
import pydub
import time
import json
import pandas as pd
import sqlite3
import os
from helper import *

In [2]:
#PATHS TO ELEMENTS

#path to first sites text
eligible_load = '//*[@id="statePods_table"]/tbody/tr[1]/td[1]'

#path to get started button
get_started = '//*[@id="main"]/section/div/div/div/button/div'

#path to enter dob
dob = '//*[@id="relationship"]/div/div[1]/div[1]/div/div[1]/div[1]/input'

#path to sex option
nb = '//*[@id="relationship"]/div/div[1]/div[2]/fieldset/div/div[3]/input'

#path to live option
live = '//*[@id="relationship"]/div/div[2]/div[1]/fieldset/div/div[1]/input'

#path to work option
work = '//*[@id="relationship"]/div/div[2]/div[2]/fieldset/div/div[2]/input'

#path to enter zip
zipcode = '//*[@id="relationship"]/div/div[3]/div/div/div[1]/div[1]/input'

#path to consent option
consent = '//*[@id="acknowledgement"]/div/section/fieldset/div/div/div/input'

#path to submit option
submit = '//*[@id="content"]/section/div[3]/div[2]/button/div'

#path to locate providers button/text
providers = '//*[@id="content"]/section/div[3]/div/button[2]/div'

#path to jb or sb sites
jb = 'jones beach'
sb = 'suny stony brook'

In [3]:
loc = jb

In [4]:
def load_page():
    search = WebDriverWait(driver, 1).until(
        EC.presence_of_element_located((By.ID, 'OuterDiv'))
    )

    results = driver.find_elements_by_xpath('//*[@id="OuterDiv"]/div')
    slots = int(results[-1].text.split()[-1])
    return slots

In [5]:
def input_field(path, value):
    
    #wait for presence of element to load
    fill_field = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located((By.XPATH, path))
    )

    #input value
    fill_field.clear()
    time.sleep(1)
    fill_field.send_keys(value)
    fill_field.send_keys(Keys.RETURN)
    time.sleep(1)

In [6]:
def click(path):
    
    #wait for presence of element to load
    box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, path))
    )
    
    #click box
    box.click()
    
    time.sleep(1)

In [7]:
def page_load(path):
    #wait for element to be loaded
    WebDriverWait(driver, 20).until(
        EC.visibility_of_element_located((By.XPATH, path)))

In [8]:
def provider(location):
    for i in range(2,4):
        providers=driver.find_elements_by_class_name('providers')
        
        num = 0
        for provider in providers:
            num+=1
            if location in provider.text.lower():
                return '//*[@id="content"]/section/section[{}]/section/section[1]/div[2]/div/p[2]/span/a'.format(num)
        
        try:
            click('//*[@id="content"]/section/section[11]/button[{}]'.format(i))
        except:
            time.sleep(2)
            click('//*[@id="content"]/section/section[11]/button[{}]'.format(i))  

In [9]:
def fill_out_forms():
    #goes to eligibilty page for ny
    eligible = 'https://am-i-eligible.covid19vaccine.health.ny.gov'
    driver.get(eligible)
        
    
    #waits for text to be loaded
    page_load(eligible_load)
    
    #click get started button to go to next page
    try:
        click(get_started)
    except:
        time.sleep(2)
        click(get_started)
    
    
    #fill in dob field
    input_field(dob, '03201935')
    
    
    #click sex option
    click(nb)
    
    
    #click yes for live in ny
    click(live)
    
    
    #click no for work in ny
    click(work)
    
    
    #input zip
    input_field(zipcode, '11777')
    
    
    #click to consent
    try:
        click(consent)
    except:
        time.sleep(2)
        click(consent)

    
    #click to submit
    try:
        click(submit)
    except:
        time.sleep(2)
        click(submit)

    
    #wait for providers page to load
    page_load(providers)
    
    #click to find providers
    click(providers)

    #find site
    site = provider(loc)
    
    #choose site
    try:
        click(site)
    except:
        time.sleep(2)
        click(site)
    
    
    #switch to new tab that is opened
    driver.switch_to.window(driver.window_handles[1])

In [10]:
def captcha():
    #waiting for captcha to load
    cap = '/html/body/app-root/recaptcha-component/div/div[2]/div/div/div/iframe'
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, cap))
    ) 
    
    #switch to new tab that is opened
    driver.switch_to.window(driver.window_handles[1])
    
    #switch to recaptcha frame
    frames=driver.find_elements_by_tag_name("iframe")
    driver.switch_to.frame(frames[0]);
    
    path = '//*[@id="recaptcha-anchor"]/div[1]'
    
    #wait for presence of element to load
    box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, path))
    )
        
    #click box
    box.click()
    
    path = '/html/body/div[3]/div[4]'
    
    #switch to recaptcha audio control frame
    driver.switch_to.default_content()
    frames=driver.find_element_by_xpath(path).find_elements_by_tag_name("iframe")
    driver.switch_to.frame(frames[0])
    
    path = '//*[@id="recaptcha-audio-button"]'
    
    #wait for presence of element to load
    box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.XPATH, path))
    )
        
    #click box
    box.click()
    
    captcha_page = driver.current_url
    
    while driver.current_url == captcha_page:
        #get the mp3 audio file
        src = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.ID, "audio-source"))
        ).get_attribute("src")
        #print("[INFO] Audio src: %s"%src)
        #download the mp3 audio file from the source
        urllib.request.urlretrieve(src, os.getcwd()+"/sample.mp3")
        
        #Convert from MP3 to WAV format
        sound = pydub.AudioSegment.from_mp3(os.getcwd()+"/sample.mp3")
        sound.export(os.getcwd()+"/sample.wav", format="wav")
        sample_audio = sr.AudioFile(os.getcwd()+"/sample.wav")
        
        sample_audio = sr.AudioFile(os.getcwd()+"/sample.wav")
        
        r= sr.Recognizer()
        
        with sample_audio as source:
            audio = r.record(source)
            
        #translate audio to text with google voice recognition
        key=r.recognize_google(audio)
        print("[INFO] Recaptcha Passcode: %s"%key)
        
        path = '//*[@id="audio-response"]'
        
        input_field(path, key)

In [ ]:
got_slot = False

# datetime object containing current date and time
now = datetime.now() # current date and time
today = now.strftime("%B %d")

while got_slot == False:
    # path to chromedriver
    PATH = '/Users/jason/Documents/github/Webdrivers/chromedriver'
    #opens an instance of chrome
    driver = webdriver.Chrome(PATH)
    
    time.sleep(5)
    
    try:
        fill_out_forms()
    except:
        driver.quit()
        continue
    
    for i in range(1000):
        #set variable sleep time
        sleep = uniform(2,4)
        
        #refresh page
        driver.refresh()

        try:
            slots = load_page()
        
        except:
            time.sleep(sleep)
            if i == 999:
                os.system('say "Restart"')
            continue
            
        works = False
        for slot in range(slots):

            try:
                valid = driver.find_element_by_xpath('//*[@id="Marker{}Div"]/div[2]/div/div[2]/div[8]/div/button'.format(slot))
                month = driver.find_element_by_xpath('//*[@id="Marker{}Div"]/div[2]/div/div[2]/div[1]'.format(slot))
                if (valid.text == 'Event Full') or (today in month.text):
                    continue
                elif month.text.split()[1] in ['February', 'March']:
                    valid.click()
                    works = True
                    break
                else:
                    continue
            except:
                continue
    
        if works == False:
            
            time.sleep(sleep)
            if i == 999:
                os.system('say "Restart"')
            continue
        else:
            try:
                tic = '//*[@id="content"]/div[4]/div[2]/div/div/table/tbody/tr[2]/td[3]/input'
                tic_box = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.XPATH, tic))
                )
                tic_box.click()
    
                next_bttn = driver.find_element_by_xpath('//*[@id="content"]/div[4]/div[2]/div/div/div[2]/fieldset/button[2]')
                next_bttn.click()
                page = True
            except:
                #close current tab
                driver.close()
                
                #switch to original tab
                driver.switch_to.window(driver.window_handles[0])
                
                #choose site
                try:
                    click(site)
                except:
                    time.sleep(2)
                    click(site)
                
                #switch to newly opened tab
                driver.switch_to.window(driver.window_handles[1])
                
                page = False
        
        if page:
            os.system('say "Vaccine time."')
            got_slot = True
            break
        elif i == 999:
            os.system('say "Restart"')
            
        time.sleep(sleep) 
    
    if got_slot:
        break
    
    driver.quit()   
    time.sleep(10) 

In [11]:
# datetime object containing current date and time
now = datetime.now() # current date and time
today = now.strftime("%B %d")


# path to chromedriver
PATH = '/Users/jason/Documents/github/Webdrivers/chromedriver'

#set chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument("--incognito")
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--profile-directory=Default')
chrome_options.add_argument("--disable-plugins-discovery")

#opens an instance of chrome
driver = webdriver.Chrome(options=chrome_options, executable_path=PATH)

#driver = webdriver.Chrome(PATH)


time.sleep(5)

try:
    fill_out_forms()
except:
    driver.quit()    


captcha()

[INFO] Recaptcha Passcode: voice recording app


In [ ]:
driver.execute_script("return navigator.userAgent")

captcha work

In [ ]:
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_experimental_option("excludeSwitches", ['enable-automation'])
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_argument("--incognito")
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--profile-directory=Default')
chrome_options.add_argument("--disable-plugins-discovery")
#chrome_options.add_argument("user-agent=Chrome/89.0.4389.114")


In [ ]:
import speech_recognition as sr
import ffmpy
import requests
import urllib
import pydub

In [ ]:
#switch to new tab that is opened
driver.switch_to.window(driver.window_handles[1])

In [ ]:
driver.current_url

In [ ]:
driver.current_url

In [44]:
#switch to new tab that is opened
driver.switch_to.window(driver.window_handles[1])

#switch to recaptcha frame
frames=driver.find_elements_by_tag_name("iframe")
driver.switch_to.frame(frames[0]);

path = '//*[@id="recaptcha-anchor"]/div[1]'

#wait for presence of element to load
box = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.XPATH, path))
)
    
#click box
box.click()

path = '/html/body/div[3]/div[4]'

#switch to recaptcha audio control frame
driver.switch_to.default_content()
frames=driver.find_element_by_xpath(path).find_elements_by_tag_name("iframe")
driver.switch_to.frame(frames[0])

path = '//*[@id="recaptcha-audio-button"]'

#wait for presence of element to load
box = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.XPATH, path))
)
        
#click box
box.click()

captcha_page = driver.current_url

while driver.current_url == captcha_page:
    #get the mp3 audio file
    src = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.ID, "audio-source"))
    ).get_attribute("src")
    #print("[INFO] Audio src: %s"%src)
    #download the mp3 audio file from the source
    urllib.request.urlretrieve(src, os.getcwd()+"/sample.mp3")
    
    #Convert from MP3 to WAV format
    sound = pydub.AudioSegment.from_mp3(os.getcwd()+"/sample.mp3")
    sound.export(os.getcwd()+"/sample.wav", format="wav")
    sample_audio = sr.AudioFile(os.getcwd()+"/sample.wav")
    
    sample_audio = sr.AudioFile(os.getcwd()+"/sample.wav")
    
    r= sr.Recognizer()
    
    with sample_audio as source:
        audio = r.record(source)
        
    #translate audio to text with google voice recognition
    key=r.recognize_google(audio)
    print("[INFO] Recaptcha Passcode: %s"%key)
    
    path = '//*[@id="audio-response"]'
    
    input_field(path, key)

[INFO] Recaptcha Passcode: manufacture those water bottles


In [ ]:
#get the mp3 audio file
src = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.ID, "audio-source"))
).get_attribute("src")
#print("[INFO] Audio src: %s"%src)
#download the mp3 audio file from the source
urllib.request.urlretrieve(src, os.getcwd()+"/sample.mp3")

#Convert from MP3 to WAV format
sound = pydub.AudioSegment.from_mp3(os.getcwd()+"/sample.mp3")
sound.export(os.getcwd()+"/sample.wav", format="wav")
sample_audio = sr.AudioFile(os.getcwd()+"/sample.wav")

sample_audio = sr.AudioFile(os.getcwd()+"/sample.wav")

r= sr.Recognizer()

with sample_audio as source:
    audio = r.record(source)
    
#translate audio to text with google voice recognition
key=r.recognize_google(audio)
print("[INFO] Recaptcha Passcode: %s"%key)

path = '//*[@id="audio-response"]'

input_field(path, key)

In [ ]:
#switch to new tab that is opened
driver.switch_to.window(driver.window_handles[1])

In [ ]:
#switch to recaptcha frame
frames=driver.find_elements_by_tag_name("iframe")
driver.switch_to.frame(frames[0]);


In [ ]:
path = '//*[@id="recaptcha-anchor"]/div[1]'

In [ ]:
#wait for presence of element to load
box = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.XPATH, path))
)
    
#click box
box.click()

In [ ]:
path = '/html/body/div[3]/div[4]'

In [ ]:
#switch to recaptcha audio control frame
driver.switch_to.default_content()
frames=driver.find_element_by_xpath(path).find_elements_by_tag_name("iframe")
driver.switch_to.frame(frames[0])


In [ ]:
path = '//*[@id="recaptcha-audio-button"]'

In [ ]:
#wait for presence of element to load
box = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.XPATH, path))
)
    
#click box
box.click()

In [ ]:
#nth iterations
path = '//*[@id=":4"]'

In [ ]:
# first iteration
path = '//*[@id=":2"]'

In [ ]:
#wait for presence of element to load
box = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.XPATH, path))
)
    
#click box
box.click()

In [ ]:
#get the mp3 audio file
src = driver.find_element_by_id("audio-source").get_attribute("src")
#print("[INFO] Audio src: %s"%src)
#download the mp3 audio file from the source
urllib.request.urlretrieve(src, os.getcwd()+"/sample.mp3")

In [ ]:
#Convert from MP3 to WAV format
sound = pydub.AudioSegment.from_mp3(os.getcwd()+"/sample.mp3")
sound.export(os.getcwd()+"/sample.wav", format="wav")
sample_audio = sr.AudioFile(os.getcwd()+"/sample.wav")

In [ ]:
sample_audio = sr.AudioFile(os.getcwd()+"/sample.wav")

In [ ]:
r= sr.Recognizer()

In [ ]:
with sample_audio as source:
    audio = r.record(source)

In [ ]:
#translate audio to text with google voice recognition
key=r.recognize_google(audio)
print("[INFO] Recaptcha Passcode: %s"%key)

In [ ]:
path = '//*[@id="audio-response"]'

In [ ]:
frames

In [ ]:
input_field(path, key)

captcha get vac

In [ ]:
driver.switch_to.alert

In [ ]:
driver.switch_to.default_content()

In [23]:
driver.switch_to.window(driver.window_handles[1])

In [ ]:
tests = driver.find_element_by_xpath('//*[@id="backEndNextContainer"]/button[1]')

In [ ]:
actions = ActionChains(driver)
actions.move_to_element('//*[@id="mat-dialog-2"]/app-second-dose-dialog/div[1]/button')
actions.click()
actions.perform()

In [62]:
test = '/html/body/app-root/app-layout/article/main/app-display-available-days/div/div/div[1]'
click(test)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=89.0.4389.114)


In [63]:
click('/html/body/div[4]/div[2]/div/mat-dialog-container/app-second-dose-dialog/div[1]/button')

In [ ]:
classes = driver.find_elements_by_xpath('/html/body/app-root/app-layout/article/main/app-display-available-times/div[1]/div/div/div/div')

In [ ]:
len(classes)

In [ ]:
classes

In [ ]:
for clas in classes:
    print(clas.text)

In [58]:
driver.switch_to.window(driver.window_handles[1])

In [61]:
#check dates in slots
appts = driver.find_elements_by_tag_name('h6')

num = 0
for appt in appts:
    num+=1
    time.sleep(2)
    print(appt.text)

try:
    click('/html/body/app-root/app-layout/article/main/app-display-available-days/div/div/div[{}]'.format(num))
except:
    time.sleep(2)
    click('/html/body/app-root/app-layout/article/main/app-display-available-days/div/div/div[{}]'.format(num))  

April 18, 2021 Sunday
May 20, 2021 Thursday
May 21, 2021 Friday
May 23, 2021 Sunday
May 24, 2021 Monday
May 25, 2021 Tuesday
May 26, 2021 Wednesday
May 27, 2021 Thursday
May 28, 2021 Friday
May 29, 2021 Saturday
May 30, 2021 Sunday
May 31, 2021 Monday


In [ ]:
<div class="btn-date rounded my-2"><div><h6 class="pl-2 pt-1 m-0">May 25, 2021 <span class="ml-2">Tuesday</span></h6><div class="d-flex flex-row justify-content-between pl-2"><div><div><span class="justify-content-end pr-2">Available appointments</span><span><b>86</b></span></div><div><label class="justify-content-end pr-2 text-nowrap m-0">Openings between</label><label class="text-nowrap"><b>8:00 AM - 6:15 PM</b></label></div></div><div class="mb-auto date-icon"><mat-icon role="img" aria-hidden="false" aria-label="Edit icon" class="mat-icon notranslate align-top material-icons mat-icon-no-color" data-mat-icon-type="font">keyboard_arrow_right</mat-icon></div></div></div></div>